In [ ]:
import numpy as np
from cv2 import imread,cvtColor,resize,COLOR_BGR2RGB
from keras.layers import Dense,Input, Reshape, LeakyReLU, Conv2D, Conv2DTranspose, Flatten, Dropout,BatchNormalization
from keras.models import Sequential
from keras.models import Model
from matplotlib import pyplot as plt
import os
from keras.utils import img_to_array
import tensorflow as tf


In [ ]:
dir='/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba'
arr=os.listdir(dir)
len(arr)

In [ ]:
pics=[]
i=0
for files in arr:
    if i>50000:
        break
    img=imread(dir+'/'+files)
    img =cvtColor(img, COLOR_BGR2RGB)
    img=resize(img,(64,64))
    img = (img - 127.5) / 127.5
    img = img.astype(float)
    pics.append(img_to_array(img))
    i=i+1
    if i%1000==0:
        print(i)

In [ ]:
def plot_images(sqr = 5):
    plt.figure(figsize = (10,10))
    plt.title("Real Images",fontsize = 35)
    for i in range(sqr * sqr):
        plt.subplot(sqr,sqr,i+1)
        plt.imshow(pics[i]*0.5 + 0.5 )
        plt.xticks([])
        plt.yticks([])

In [ ]:
plot_images(6)

In [ ]:
latent_dim = 100
generator=Sequential()
generator.add(Dense(128*128*3, use_bias=False, input_shape=(latent_dim,)))
generator.add(Reshape((128,128,3)))
generator.add(Conv2D(128,4,padding='same',use_bias=False,strides=1))
generator.add(Conv2D(128,4,padding='same',use_bias=False,strides=2))
generator.add(BatchNormalization())
generator.add(LeakyReLU())
generator.add(Conv2D(256,4,padding='same',use_bias=False,strides=1))
generator.add(Conv2D(256,4,padding='same',use_bias=False,strides=2))
generator.add(BatchNormalization())
generator.add(LeakyReLU())
generator.add(Conv2D(512,4,padding='same',use_bias=False,strides=1))
generator.add(Conv2D(512,4,padding='same',use_bias=False,strides=2))
generator.add(BatchNormalization())
generator.add(LeakyReLU())
generator.add(Conv2DTranspose(256, 4,padding='same', strides=1,kernel_initializer='he_normal',use_bias=False))
generator.add(Conv2DTranspose(256, 4,padding='same', strides=2,kernel_initializer='he_normal',use_bias=False))
generator.add(BatchNormalization())
generator.add(Conv2DTranspose(128, 4,padding='same', strides=1,kernel_initializer='he_normal',use_bias=False))
generator.add(Conv2DTranspose(128, 4, padding='same',strides=2,kernel_initializer='he_normal',use_bias=False))
generator.add(BatchNormalization())
#generator.add(Conv2DTranspose(2, 4, strides=2,kernel_initializer='he_normal',use_bias=False,activation = 'tanh'))

In [ ]:
generator.summary()

In [ ]:
discriminator=Sequential()
discriminator.add(Input((64,64,3)))
discriminator.add(Conv2D(128,4, strides=2, padding='same',kernel_initializer='he_normal', use_bias=False))
discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU())
discriminator.add(Conv2D(128,4, strides=2, padding='same',kernel_initializer='he_normal', use_bias=False))
discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU())
discriminator.add(Conv2D(256,4, strides=2, padding='same',kernel_initializer='he_normal', use_bias=False))
discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU())
discriminator.add(Conv2D(256,4, strides=2, padding='same',kernel_initializer='he_normal', use_bias=False))
discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU())
discriminator.add(Conv2D(512,4, strides=2, padding='same',kernel_initializer='he_normal', use_bias=False))
discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU())
discriminator.add(Flatten())
discriminator.add(Dense(1,activation='sigmoid'))

In [ ]:
discriminator.summary()

In [ ]:
optimizer = tf.keras.optimizers.legacy.RMSprop(
        lr=.0001,
        clipvalue=1.0,
        decay=1e-8
    )
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits = True)

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output),fake_output)
def discriminator_loss(fake_output, real_output):
    fake_loss = cross_entropy(tf.zeros_like(fake_output),fake_output)
    real_loss = cross_entropy(tf.ones_like(real_output),real_output)
    return fake_loss + real_loss

In [ ]:
latent_dim=100
def train_steps(images):
    noise = np.random.normal(0,1,(batch_size,latent_dim))
    with tf.GradientTape() as gen_tape , tf.GradientTape() as disc_tape:
        generated_images = generator(noise)
        fake_output = discriminator(generated_images)
        real_output = discriminator(images)
        
        gen_loss = generator_loss(fake_output)
        dis_loss = discriminator_loss(fake_output, real_output)
        
        
    gradient_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)    
    gradient_of_discriminator = disc_tape.gradient(dis_loss, discriminator.trainable_variables)
    
    optimizer.apply_gradients(zip(gradient_of_generator,generator.trainable_variables))
    optimizer.apply_gradients(zip(gradient_of_discriminator, discriminator.trainable_variables))
    
    loss = {'gen loss':gen_loss,
           'disc loss': dis_loss}
    return loss

In [ ]:
import time
def train(epochs,dataset):
    
    for epoch in range(epochs):
        start = time.time()
        print("\nEpoch : {}".format(epoch + 1))
        for images in dataset:
            loss = train_steps(images)
        print(" Time:{}".format(np.round(time.time() - start),2)) 
        print("Generator Loss: {} Discriminator Loss: {}".format(loss['gen loss'],loss['disc loss']))

In [ ]:
batch_size=32
dataset=tf.data.Dataset.from_tensor_slices(np.array(pics)).batch(batch_size)
train(50,dataset)

In [ ]:
def plot_generated_images(square = 5, epochs = 0):
    plt.figure(figsize = (10,10))
    for i in range(square * square):
        if epochs != 0:    
            if(i == square //2):
                plt.title("Generated Image at Epoch:{}\n".format(epochs), fontsize = 32, color = 'black')
        plt.subplot(square, square, i+1)
        noise = np.random.normal(0,1,(1,latent_dim))
        img = generator(noise)
        plt.imshow(np.clip((img[0,...]+1)/2, 0, 1))
        plt.xticks([])
        plt.yticks([])
        plt.grid()

In [ ]:
plot_generated_images(1)


In [ ]:
plot_generated_images(4)


In [ ]:
plot_generated_images(6)
